In [6]:
import talos as ta
from talos.metrics.keras_metrics import fmeasure

import pandas as pd
import numpy as np

# this a locally modified version of the actual package
from livelossplot import PlotLossesKeras

from keras.models import Sequential
from keras.layers import Dropout, Dense

# Keras items
from keras.optimizers import Adam, Nadam
from keras.activations import relu, elu, sigmoid
from keras.losses import binary_crossentropy

# 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, f1_score

# 
from keras_tqdm import TQDMNotebookCallback

# 
from imblearn.metrics import geometric_mean_score
#########################################################################

%matplotlib inline
pd.set_option("max_columns",90, "max_rows", 90)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys 
sys.path.insert(0, 'C:\\Users\\qzhang\\PycharmProjects\\Mei')
from history.utility import model_fit

from talos.model import lr_normalizer, early_stopper, hidden_layers

def data_prep(raw_df):
    """"""
    df = raw_df.copy()
    
    JUD_ST = ('CT', 'DE', 'FL', 'IL', 'IN', 'KS', 'KY', 'LA', 'ME', 'MA',
              'NE', 'NJ', 'NM', 'NY', 'ND', 'OH', 'OK', 'PA', 'SC', 'SD',
              'VT', 'WI')
    CATE_VAR = ['orig_chn', 'loan_purp', 'prop_type', 'occ_stat', 'judicial_st', 'fhb_flag']

    # drop columns
    df.drop(['loan_id', 'status_prev', 'msa'], axis=1, inplace=True)

    # drop all the observation with missing value
    df.dropna(how='any', inplace=True)

    # create a new feature based on prop_state
    df.loc[:, 'judicial_st'] = df['prop_state'].apply(lambda x: 'Y' if x in JUD_ST else 'N')
    df.drop(['prop_state'], axis=1, inplace=True)

    # convert status to 0 or 1
    df.loc[:, 'status'] = df['status'].apply(lambda x: int(x == 'D60-D90'))

    X = df.drop(['status'], axis=1).copy()
    num_feats = list(set(X.columns) - set(CATE_VAR))
    X = pd.get_dummies(X, columns=CATE_VAR)
    y = df['status'].copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

    # mean is computed only over the training data and then subtracted equally from all splits(train/val/test)
    scaler = StandardScaler()
    X_train.loc[:, num_feats] = scaler.fit(X_train[num_feats]).transform(X_train[num_feats])
    X_test.loc[:, num_feats] -= scaler.mean_
    X_test.loc[:, num_feats] /= scaler.scale_
    return X_train, X_test, y_train, y_test

# read data
raw_df = pd.read_csv('..\Data\Gse_panel_current_sample_raw.csv')


In [ ]:
def neural_nets(X_train, y_train, X_val, y_val, params):
    model = Sequential()
    model.add(Dense(params['first_neuron'], 
                    input_dim=X_train.shape[1],
                    activation=params['activation'],
                    kernel_initializer=params['kernel_initializer']))
    
    model.add(Dropout(params['dropout']))

    hidden_layers(model, params, 1)

    model.add(Dense(1, activation=params['last_activation'],
                    kernel_initializer=params['kernel_initializer']))
    
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'](),
                  metrics=['acc'])
    
    history = model.fit(X_train, y_train, 
                        validation_data=[X_val, y_val],
                        batch_size=params['batch_size'],
                        callbacks=[],  # , TQDMNotebookCallback() PlotLossesKeras()
                        epochs=params['epochs'],
                        verbose=0)

    # predict 
    prob_pos = model.predict(X_val, batch_size=params['batch_size'], verbose=0) 
    vfunc = np.vectorize(lambda x: 1 if x > 0.05 else 0)
    y_pred = vfunc(prob_pos)

    # calculate metrics
    precision, recall, _ = precision_recall_curve(y_val, prob_pos) 
    fpr, tpr, _ = roc_curve(y_val, prob_pos)
    roc_auc = auc(fpr, tpr)
    pr_auc = average_precision_score(y_val, prob_pos)
    exp_pos = np.sum(prob_pos)
    f_score = f1_score(y_val, y_pred)
    g_mean = geometric_mean_score(y_val, y_pred)

    op = {'PR_AUC': pr_auc,
          'ROC_AUC': roc_auc,
          'F score': f_score,
          'G mean': g_mean,
          'Expeted num of D60-D90': exp_pos,
          'Actual num of D60-D90': np.sum(y_val),}
    print(model.summary())
    return history, model, op

In [7]:
# split sample set
X, X_test, y, y_test = data_prep(raw_df)


In [8]:
# p = {'lr': [10**i for i in range(-6, 1)],
#      'first_neuron': (4, 46, 4),
#      # 'hidden_layers': [0, 1, 2, 3],
#      'batch_size': [2**i for i in range(2, 13)],
#      'epochs': (5, 100, 5),
#      'dropout': (0, 0.5, 5),
#      'kernel_initializer': ['uniform','normal'],
#      'optimizer': [Nadam, Adam],
#      'losses': [binary_crossentropy],
#      'activation': [relu],
#      'last_activation': [sigmoid]}

p = {'lr': [2, 1],
     'first_neuron': [5],
     'hidden_layers': [1],
     'batch_size': [2000],
     'shapes': ['funnel'],
     'epochs': [1],
     'dropout': [0],
     'kernel_initializer': ['uniform'],
     'optimizer': [Adam],
     'losses': [binary_crossentropy],
     'activation': [relu],
     'last_activation': [sigmoid]}


In [9]:
t = ta.Scan(x=X.values,
            y=y.values,
            val_split=.2,
            model=neural_nets,
            grid_downsample=None, 
            params=p,
            search_method='random',
            dataset_name='Mei_NN_HPT',
            experiment_no='50',
            debug = False)  # 

C:\Users\qzhang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\qzhang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 5)                 235       
_________________________________________________________________
dropout_2 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 30        
_________________________________________________________________
dropout_3 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6         
Total params: 271
Trainable params: 271
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\qzhang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\qzhang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 5)                 235       
_________________________________________________________________
dropout_4 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 30        
_________________________________________________________________
dropout_5 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 6         
Total params: 271
Trainable params: 271
Non-trainable params: 0
_________________________________________________________________
None
2 scans will take roughly 36 seconds


NotFoundError: FeedInputs: unable to find feed output dense_3_target:0